In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage import io, transform, color
from colorize import network, util

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

print('Device:', device)

In [ ]:
train_images = glob.glob("data/**/*.jpg")[:-10]
val_images = glob.glob("data/**/*.jpg")[-10:]

# Show some random images
for path in np.random.choice(train_images, 3, replace=False):
    L, ab = util.imread(path)
    
    # Show encoded/decoded
    Y = transform.resize(ab, (56, 56))
    Z = util.soft_encode(Y)
    Y_decoded = util.decode(Z)
    Y_decoded = transform.resize(Y_decoded, (224, 224))
    
    util.side_by_side(L, ab, L, Y_decoded)

In [ ]:
# Train model
net = network.Network()
net(torch.zeros((1, 1, 224, 224)), summary=True)
util.train(net, train_images, device, epochs=5)

In [ ]:
# Load trained model
net = network.Network()
net.load_state_dict(torch.load('models/model_1_full_876b954262efd3be6621e91ef6b99a6e6dff7cc9.pth', map_location=device))

In [ ]:
# Colorize the validation images
util.colorize_images(net, val_images, device)